In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
sabahesaraki_2017_2017_path = kagglehub.dataset_download('sabahesaraki/2017-2017')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install tensorflow keras opencv-python matplotlib transformers

#ImAge Feature Extraction using Pre-Trained CNN:


*I will be making Use  Of RESNET50(50 Layers)

In [ ]:
import os
import json
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
import numpy as np

# Paths to the image and annotations
image_folder =
annotations_path =

# Loading captions JSON File
with open(annotations_path, 'r') as f:
  annotations =json.load(f)

# Creating a dictionary to map image IDs to captions
captions_dict = {}
for annotation in annotations:
  image_id = annotation['image_id']
  caption = annotation['caption']
  if image_id not in captions_dict:
    captions_dict[image_id]= []
  captions_dict[image_id].append(caption)



## Extracting Features from All Images in the folder Using ResNet50


In [ ]:
## Loading the Resnet50 without the top layer for feature extraction
base_model = ResNet50(weights='imagenet', include_top='False', pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

def extract_features(img_path):
  img = image.load_img(img_path, target_size=(224, 224)) ## Resizing to 224 X 224
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array = preprocess_input(img_array) ## Preprocessing the image in ResNet50
  features = model.predict(img_array,verbose=0)
  return features


## Extract Features Of all images
image_features= {}
for img_file in os.listdir(image_folder):
  img_id = int(img_file.split('.')[0]) # Get immage ID from file name
  img_path = os.path.join(image_folder, img_file)
  features = extract_features(img_path)
  image_features[img_id] = features

## Check extracted features
print(f"Extracted features for {len(image_features)} images.")


## 4: Prepare Captions For Training
Tokenizing the captions so as to feed them into the Long short Term memory(LSTM) or Transformer


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Get all captions
all_captions = [ caption for captions in captions_dict.values() for caption in captions]

## Tokenize Captions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)
vocab_size = lent(tokenizer.word_index)+ 1

# Convert Captions to Sequences
max_length = max(len(caption.split())for caption in all_captions)
sequences = tokenizer.texts_to_sequences(all_captions)
padded_sequences =pad_sequences(sequences, maxlen=max_length, padding='post')

print(f"Vocabulary_size: {voca_size}")
print(f"Max Caption length: {max_length}")




#5 :Building the Image Captioning Model (CNN + LSTM)
Building a simple CNN And LOng Short term Memory Model for image Captioning


In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate
from tensorflow.keras.models import Model

# Feature input (image features from ResNet50)
image_input = Input(shape=(2048,))
image_dense = Dense(256, activation='relu')(image_input)

## Captioning Input
caption_input = Input(shape=(max_length,))
caption_emb = Embedding(vocab_size, 256)(caption_input)
caption_lstm = LSTM(256)(caption_emb)

## Combine image features and captions
decoder = Concatenate()([image_dense, caption_lstm])
output = Dense(vocab_size, activation='softmax')(decoder)

# Build the Model
model = Model(inputs = [image_input, caption_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy')
print(model.summary())

### TRaining batches of image features and Captions

In [1]:
for img_id captions in captions_dict.items():
  image_feat = image_features[img_id]
  for caption in captions:
    seq = tokenizer.texts_to_sequences([caption])[0]
    for i in range(1, len(seq)):
      input_seq, output_word = seq[:1], seq[i]
      input_seq = pad_sequences([input_seq], maxlen=max_length)[0]
      output_word = np.zeros(vocab_size)
      output_word[seq[i]]=1
      model.train_on_batch([image_feat, input_seq], output_word)


### Generating the Caption For a New Image

In [ ]:
def generate_caption(image_features, tokenizer, max_length):
  input_text = '<start>'
  for i in range(max_length):
    seq = tokenizer.texts_to_sequences([input_text])[0]
    seq = pad_sequences([seq], maxlen=max_length)
    prediction = model.predict([image_features,seq], verbose=0)
    predicted_word = tokenizer.index_word[np.argmax(prediction)]
    input_text += ' ' + predicted_word
    if predicted_word == '<end>':
      break
return input_text


#### Testing The Image Caption Model

In [ ]:

image_id= list(image_features.keys())[0]
print(generate_caption(image_features[image_id],tokenizer, max_length))